In [1]:
import requests, json, numpy, datetime
import pywren
import boto3
import numpy as np

In [2]:
GEOJSONS = ['train-borde_rural.geojson', 'train-borde_soacha.geojson',
            'train-castries.geojson', 'train-dennery.geojson',
            'train-gros_islet.geojson', 'train-mixco_1_and_ebenezer.geojson',
            'train-mixco_3.geojson']

In [3]:
BUCKET = 'mapping-disaster-risk'

TIFS = ['borde_rural_ortho-cog.tif', 'borde_soacha_ortho-cog.tif',
        'castries_ortho-cog.tif', 'dennery_ortho-cog.tif',
        'gros_islet_ortho-cog.tif', 'mixco_1_and_ebenezer_ortho-cog.tif',
        'mixco_3_ortho-cog.tif']

# Note that all geojsonn names are the object names as well, so to access 
# call: https://mapping-disaster-risk.s3.amazonaws.com/train-borde_rural.geojson

In [17]:
# limited to a single name of geojson file passed in

def get_geojsons(geojson):
    '''
    Load geojson, extract relevant information, return as a dictionary.
    '''
    try:
        s3 = boto3.client('s3', 'us-east-1')
        obj = s3.get_object(Bucket='mapping-disaster-risk', Key=geojson)
        geo_json = json.load(obj['Body'])
        polygons = []
        for feature in geo_json['features']:
            polygon = {}
            polygon['type'] = feature['geometry']
            polygon['bid'] = feature['id']
            polygon['roof_material'] = feature['properties']['roof_material']
            polygon['coordinates'] = feature['geometry']['coordinates']
            polygon['geojson'] = geojson
            polygons.append(polygon)
    except Exception as e:
        return e

    return polygons

In [18]:
# Obtain footprint dictionaries
# Returns a list of lists, where each list are polygons for one .tif file
pwex = pywren.default_executor()
futures = pwex.map(get_geojsons, GEOJSONS)
footprints = pywren.get_all_results(futures)

In [19]:
# This is the first .tif file's footprints
footprints[0]

[{'type': {'type': 'Polygon',
   'coordinates': [[[-74.15872901622859, 4.55513208404803],
     [-74.15869565550265, 4.555234421743102],
     [-74.15864296010655, 4.555217122695224],
     [-74.15867632083824, 4.555114785003784],
     [-74.15872901622859, 4.55513208404803]]]},
  'bid': '7a3f2a10',
  'roof_material': 'concrete_cement',
  'coordinates': [[[-74.15872901622859, 4.55513208404803],
    [-74.15869565550265, 4.555234421743102],
    [-74.15864296010655, 4.555217122695224],
    [-74.15867632083824, 4.555114785003784],
    [-74.15872901622859, 4.55513208404803]]],
  'geojson': 'train-borde_rural.geojson'},
 {'type': {'type': 'Polygon',
   'coordinates': [[[-74.15879484827516, 4.555159047202581],
     [-74.15876112811927, 4.555251495820111],
     [-74.15870434247019, 4.555230637593402],
     [-74.15873806263139, 4.555138188979648],
     [-74.15879484827516, 4.555159047202581]]]},
  'bid': '7a1f731e',
  'roof_material': 'irregular_metal',
  'coordinates': [[[-74.15879484827516, 4.555